# Predicting credit card fraud with autoencoders

Credit card fraud is a complicated problem:

- The data set is hugely imbalanced.
- Low quality of labeled data => fraud patterns hard to find.

We will give it a try to a neural network technique called *autoencoders*.

Originally used for computer vision, autoencoders are 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/creditcard.csv")

FileNotFoundError: File b'./data/creditcard.csv' does not exist

In [ ]:
df.head()

In [ ]:
df.groupby(['Class'])['Class'].count()

We have 492 fraudulent transactions against 28k...

In [ ]:
%matplotlib inline
df.groupby(['Class'])['Class'].hist()

In [ ]:
fraud = df[df['Class']==1]
normal = df[df['Class']==0]

In [ ]:
normal['Amount'].hist()

In [ ]:
fraud['Amount'].hist()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scl = StandardScaler()

In [ ]:
normal['Amount'] = scl.fit_transform(normal['Amount'].values.reshape(-1,1))

In [ ]:
normal.head()

In [ ]:
X = normal.drop(['Class','Time'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test = train_test_split(X, test_size=0.2)

## Building the model

Let's use `keras` to create our autoencoder network.


In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
X_train.shape

In [ ]:
input_dim = X_train.shape[1]

In [ ]:
inner_dim = int(input_dim/2)

In [ ]:
inner_dim

In [ ]:
input_layer = Input(shape=(input_dim,))

In [ ]:
encoder = Dense(inner_dim, activation = 'tanh')(input_layer)

In [ ]:
decoder = Dense(input_dim, activation='relu')(encoder)

In [ ]:
autoencoder = Model(inputs = input_layer, outputs=decoder)

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
history = autoencoder.fit(X_train, X_train, 
                         epochs = 2, 
                         batch_size = 32).history

In [ ]:
import numpy as np

In [ ]:
fraud['Amount'] = scl.transform(fraud['Amount'].values.reshape(-1,1))

In [ ]:
true_fraud = fraud['Class'].values

In [ ]:
X_fraud = fraud.drop(['Class','Time'], axis=1)

In [ ]:
preds = autoencoder.predict(X_fraud)

In [ ]:
mse = np.mean(np.power(X_fraud-preds,2), axis=1)

In [ ]:
mse.describe()

In [ ]:
threshold = 0.4

In [ ]:
suspicious = np.where(mse>=threshold, 1, 0)

In [ ]:
suspicious

In [ ]:
np.sum(suspicious!=true_fraud)/len(true_fraud)*100